In [1]:
# open file 
mni = open("input.txt","r");
hex_bytes = readline(mni)
bits_array = bitstring.([parse(UInt16,hex_bytes[i], base=16) for i in 1:length(hex_bytes)])
bits = ""
for b in bits_array
    tail = b[end-3:end]
    bits *= tail;
end

In [2]:
function VVV(bits, c)
    return parse(Int, bits[c:c+2], base=2)
end
function TTT(bits, c)
    return parse(Int, bits[c+3:c+5], base=2)
end
function sp_type(bits, c)
    return parse(Bool, bits[c+6]) ? "Packets" : "Bits"
end
function LLLL(bits, c)
    if TTT(bits, c) == 4
        return -1
    end
    
    if sp_type(bits, c) == "Packets" 
        return parse(Int, bits[c+7:c+7+10], base=2)
    elseif sp_type(bits, c) == "Bits" 
        return parse(Int, bits[c+7:c+7+14], base=2)
    end
end
function A_idx(bits, c)
    if sp_type(bits, c) == "Packets" 
        return c+18
    elseif sp_type(bits, c) == "Bits" 
        return c+22
    end
end
function get_node_literal(bits, c)
    idx = c+6
    bitstr = ""
    while bits[idx] == '1'
        bitstr *= bits[idx+1:idx+4]
        idx += 5
    end
    bitstr *= bits[idx+1:idx+4]
    idx += 5
    return parse(Int, bitstr, base=2), idx
end

get_node_literal (generic function with 1 method)

In [3]:
function packet_parse(bits, c)
    if !occursin('1', bits[c:end])
        return ("Empty"), -1
    end
    
    V = VVV(bits, c)
    T = TTT(bits, c)
       
    if T == 4
        literal, next = get_node_literal(bits, c)
        return (V, T, literal), next
    else
        I = sp_type(bits, c)
        L = LLLL(bits, c)     
        subnodes = []
        next = A_idx(bits, c)
        if I == "Packets"
            for i in 1:L
                subnode, next = packet_parse(bits, next)
                push!(subnodes, subnode)
            end
        elseif I == "Bits"
            packet_end = next + L
            while next < packet_end
                subnode, next = packet_parse(bits, next)
                push!(subnodes, subnode)
                if subnode[1] == "Empty"
                    break
                end
            end
        end
        return (V, T, I, L, subnodes), next
    end
end

packet_parse (generic function with 1 method)

In [4]:
function get_val_sum(packet)
    val_sum = packet[1]
    if packet[2] != 4
        for subpacket in packet[5]
            val_sum += get_val_sum(subpacket)
        end
    end
    
    return val_sum
end

get_val_sum (generic function with 1 method)

In [5]:
function get_packet_val(packet)
    T = packet[2]
    if  T == 0
        return sum(sp->get_packet_val(sp), packet[5])
    elseif T == 1
        return prod(sp->get_packet_val(sp), packet[5])
    elseif T == 2
        return findmin([get_packet_val(sp) for sp in packet[5]])[1]
    elseif T == 3
        return findmax([get_packet_val(sp) for sp in packet[5]])[1]
    elseif T == 4
        return packet[3]
    elseif T == 5
        return get_packet_val(packet[5][1]) > get_packet_val(packet[5][2]) ? 1 : 0
    elseif T == 6
        return get_packet_val(packet[5][1]) < get_packet_val(packet[5][2]) ? 1 : 0
    elseif T == 7
        return get_packet_val(packet[5][1]) == get_packet_val(packet[5][2]) ? 1 : 0
    end
end

get_packet_val (generic function with 1 method)

In [6]:
a, next = packet_parse(bits, 1)

((5, 0, "Packets", 53, Any[(5, 4, 211), (6, 0, "Bits", 51, Any[(4, 4, 11312662170)]), (2, 3, "Packets", 2, Any[(4, 4, 230), (3, 4, 9149058)]), (7, 1, "Bits", 76, Any[(4, 4, 18660), (5, 7, "Packets", 2, Any[(6, 4, 6), (1, 4, 294)])]), (0, 0, "Bits", 48, Any[(4, 4, 64337), (5, 4, 10), (3, 4, 5)]), (1, 1, "Packets", 2, Any[(2, 6, "Bits", 32, Any[(5, 4, 60), (3, 4, 60)]), (7, 4, 6475254)]), (6, 4, 630700), (3, 3, "Bits", 26, Any[(1, 4, 13722)]), (0, 4, 4492), (7, 1, "Packets", 2, Any[(2, 4, 12), (7, 6, "Bits", 32, Any[(2, 4, 197), (3, 4, 197)])])  …  (1, 1, "Bits", 32, Any[(7, 4, 244), (6, 4, 194)]), (0, 0, "Packets", 5, Any[(6, 4, 142), (4, 4, 11161), (4, 4, 247), (5, 4, 2618), (4, 4, 16293547)]), (7, 1, "Packets", 2, Any[(2, 6, "Bits", 110, Any[(1, 0, "Bits", 33, Any[(2, 4, 7), (2, 4, 3), (1, 4, 3)]), (6, 0, "Bits", 33, Any[(6, 4, 9), (1, 4, 14), (0, 4, 3)])]), (3, 4, 929778)]), (3, 0, "Bits", 82, Any[(0, 4, 404241717), (1, 4, 10324282)]), (1, 2, "Packets", 5, Any[(1, 4, 2884), (4, 4, 11

In [7]:
a_val_sum = get_val_sum(a)

927

In [8]:
get_packet_val(a)

1725277876501